In [1]:
import pandas as pd
import numpy as np

One tailed test

H0: New machine packs faster

H1 : New machine doesn't pack faster

In [10]:
machine = pd.read_csv('files_for_lab/machine.txt', delimiter='\t', encoding='utf-16le')

In [11]:
machine.head()

,New machine,Old machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5


In [22]:
machine.columns

Index(['New machine', '    Old machine'], dtype='object')

In [24]:
from scipy.stats import ttest_rel

t_statistic, p_value = ttest_rel(machine['New machine'], machine['    Old machine'])
alpha = 0.05

t_statistic, p_value


(-3.0614273841115844, 0.013540335651632521)

As p_value 0.013 < alpha 0.05

Reject the null hypothesis.
The new machine packs cartons faster on average than the current machine.

Matched Pairs Test 

In [25]:
pokemon = pd.read_csv('files_for_lab/pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [28]:
t_statistic, p_value = ttest_rel(pokemon['Defense'], pokemon['Attack'])

alpha = 0.05

t_statistic, p_value


(-4.325566393330478, 1.7140303479358558e-05)

As p_value 1.7140303479358558e-05 < alpha 0.05

Reject the null hypothesis.
There is a statistically significant difference between defense and attack scores.

Setting up ANOVA

In [29]:
anova= pd.read_excel('files_for_lab/anova_lab_data.xlsx')
anova.head()

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


In [35]:
anova.shape

(15, 2)

In [32]:
anova.columns

Index(['Power ', 'Etching Rate'], dtype='object')

In [33]:
anova['Power '].value_counts()

160 W    5
180 W    5
200 W    5
Name: Power , dtype: int64

In [34]:
anova.groupby(['Power ']).agg({"Etching Rate":np.mean})

,Etching Rate
Power,
160 W,5.792
180 W,6.238
200 W,8.318


H0: Changing the power of the plasma beam has no effect on the mean etching rate.

H1: Changing the power of the plasma beam has effect on the mean etching rate.

alpha = 0.05

Degrees of freedom: m-1 where m is number of groups being compared = 3-1 = 2
Error: n-m = 15 - 2 = 13
Total DoF: n-1 where  n is the number of datapoints collected n - 1 = 15 - 1 = 14

In [39]:
S2t = 0
for power in anova['Power '].unique():
    ng = len(anova[anova['Power '] == power])  
    S2t  += ( ( anova[anova['Power '] == power]['Etching Rate'].mean() - anova['Etching Rate'].mean() ) ** 2) * ng
S2t /= ( anova['Power '].nunique() - 1 )
print("The value of S2t is {:.2f}".format(S2t)) 

The value of S2t is 9.09


In [42]:
S2E = 0
for power in anova['Power '].unique():
    for rate in anova[anova['Power '] == power]['Etching Rate']:
        S2E += (rate - anova[anova['Power '] == power]['Etching Rate'].mean()) ** 2
S2E /= (len(anova) - anova['Power '].nunique())

print()
print("The value of S2E is {:.2f}".format(S2E))



The value of S2E is 0.25


In [43]:
F = S2t / S2E
print("The value of F is {:.2f}".format(F))

The value of F is 36.88


In [49]:
from scipy import stats

p_value = stats.f_oneway(anova[anova['Power '] == '160 W']['Etching Rate'], anova[anova['Power '] == '180 W']['Etching Rate'], anova[anova['Power '] == '200 W']['Etching Rate'])[1]
p_value

7.506584272358903e-06

As the p_value is < 0.05 we reject H0. Changing the power of the plasma beam has effect on the mean etching rate.